In [1]:
import sys, os
try:
    sys.path.append('../AutoDiff')
    from variables import Variable
except:
    from AutoDiff.variables import Variable
try:
    
    from Optimizer import minimize
except:
    from Implementation.Optimizer import minimize

    
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [2]:
def plot_path_2D(val_arr,x_grid,y_grid,f_grid,title):
    

    plt.contourf(x_grid, y_grid, f_grid, cmap='Blues',
                    norm=colors.LogNorm(vmin=f_grid.min(), vmax=f_grid.max())
                    )
    plt.colorbar(orientation='horizontal')

    v0 = val_arr[0]
    vends=val_arr[-1]
    print(v0)
    plt.plot(val_arr[:, 0], val_arr[:, 1], label='path',color='black',
                linewidth=3.0)

    plt.scatter(v0[0], v0[1], s=100, color='red', label='start')
    plt.scatter(vends[0], vends[1], s=100, color='black', label='minimum')

    plt.legend(fontsize=14)

    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Start = [{0}, {1}]'.format(v0[0], v0[1]), fontsize=16)

    plt.xlim([x_grid.min(), x_grid.max()])
    plt.ylim([y_grid.min(), y_grid.max()])

In [3]:
def f1(x, y): return 100*(y-x**2)**2 + (1-x)**2
res = minimize(f1, [-1,1], method="BFGS", precision=1e-5,max_iter=10000)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
x_grid = np.linspace(-3, 3, 150)
y_grid = np.linspace(-3, 4, 200)
f_grid = f1(x_grid.reshape(1, -1),
           y_grid.reshape(-1, 1))
res.val_rec

In [ ]:
from scipy.optimize import fmin
fmin(f, [3,2,3])

In [ ]:

plot_path_2D(res.val_rec,x_grid,y_grid,f_grid,"BFGS")


In [ ]:
def min_gradientdescent(fn, x0, precision, max_iter, lr=0.01):
     # create initial variables
    # right now we only test with the 26 alphabets
    from string import ascii_lowercase
    import time
    import numpy as np

    name_ls = iter(ascii_lowercase)

    # create initial variables
    var_names = []
    for i in x0:
        name = next(name_ls)
        var_names.append(name)

    x = np.array(x0)
    s = 0 # initialize as 0 works to ensure that s=g in 1st iteration

    nums_iteration = 0
    val_rec = []
    time_rec = []
    init_time = time.time()
     # initial guess of n = 0.01
    n = 0.01
    while True:
        # recreate new variables with new values
        x_var = []
        for i, v in enumerate(x):
            x_var.append(Variable(var_names[i], v))
        # obtain values and jacobian to find delta_f
        val_vector = np.array([value.val for value in x_var])
        jacobian = np.array([fn(*x_var).der.get(i) for i in var_names])
        delta_f = jacobian*val_vector


        # update x
        old_x = x
        x = x - lr*delta_f
        print(x)
        # threshold stopping condition
        if max(abs(x-old_x)) < precision:
            return Result(x, val_rec, time_rec, True)

        # store history of values
        val_rec.append(x)

        time_rec.append(time.time()-init_time)

        # iteration stopping condition
        if nums_iteration >= max_iter:
            return Result(x, val_rec    , time_rec, False)
        nums_iteration +=1


In [ ]:
def min_steepestdescent(fn, x0, precision, max_iter):
     # create initial variables
    # right now we only test with the 26 alphabets
    from string import ascii_lowercase
    import time
    import numpy as np
    from scipy.optimize import fmin

    name_ls = iter(ascii_lowercase)

    # create initial variables
    var_names = []
    for i in x0:
        name = next(name_ls)
        var_names.append(name)

    x = np.array(x0)
    s = 0 # initialize as 0 works to ensure that s=g in 1st iteration

    nums_iteration = 0
    val_rec = []
    time_rec = []
    init_time = time.time()
     # initial guess of n = 0.01
    n = 0.01
    while True:
        # recreate new variables with new values
        x_var = []
        for i, v in enumerate(x):
            x_var.append(Variable(var_names[i], v))
        # obtain values and jacobian to find delta_f
        val_vector = np.array([value.val for value in x_var])
        jacobian = np.array([fn(x_var).der.get(i) for i in var_names])
        delta_f = jacobian*val_vector


        find_min = fmin(fn, val_vector-n*delta_f, maxiter = 1, disp=False)
        n = (find_min - x)/delta_f

        # update x
        old_x = x
        x = x + n*delta_f

        # threshold stopping condition
        if max(abs(x-old_x)) < precision:
            return Result(x, val_rec, time_rec, True)

        # store history of values
        val_rec.append(x)

        time_rec.append(time.time()-init_time)

        # iteration stopping condition
        if nums_iteration >= max_iter:
            return Result(x, val_rec    , time_rec, False)
        nums_iteration +=1
